In [3]:
# Source Libraries
source('src/timeseries_analysis.r')


#Read in Data
ds_sbc <- read_feather('sbc.fthr')

#Set clock to zero
ds_sbc$time = ds_sbc$time - 1473369000
colnames(ds_sbc)[colnames(ds_sbc)=="time"] <- "t"

#Subsample from the dataset
sample_times = seq(0,5*24*3600,2*3600)
ds <- ds_sbc[ds_sbc$t %in% sample_times,]
head(ds)

#Ocean profile
stratdf <- read_feather('sbc_strat.fthr')
strat   <- stratdf$strat
depth   <- stratdf$depth

#Subsamples Stratification
ss <- seq(1,length(strat),45)
strat <- strat[ ss ]
depth <- depth[ ss ]

#Make a initial parameter space
source('../src/inversion.r')
source('../src/modes.r')

#Make parameter space

#Coherent freq's in cph
coh_freqs = c( 0.79822616, 
               1.19733925,  
               1.99556541 ,  
               8.7804878 ,  
               60.26607539,
               64.25720621 , 
               96.18625277, 
               107.76053215, 
               205.94235033) 

#coh_freqs <- coh_freqs/(3600)
#omega <- seq(coh_freqs[3],coh_freqs[8],length.out=100) 
M2 <- 0.0805
modes <- seq(1,4,1)
#omega <- c(M2,5*M2)/3600
omega <- c(seq(1,5)*M2/3600,1.5/(24*3600))
headings  <- c(0)
k <- generate_wavenumbers(depth,strat,omega,modes)
headings <- seq(0,360,5)
ps <- generate_parameter_space(omega,modes,k,headings) 



t,z,temp,t_mean,s_mean,dt/dz,dT,dz,x,y
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,9.420437,16.010,16.40634,33.45622,-0.1918272,-0.3963379797,2.06612025,0.4780085,1.734641
0,10.910527,15.724,16.17577,33.44632,-0.1859820,-0.4517731109,2.42912257,0.4780085,1.734641
0,12.985729,15.509,15.84799,33.44376,-0.1777103,-0.3389933037,1.90756095,0.4780085,1.734641
0,15.922649,15.485,15.48471,33.43944,-0.1420410,0.0002925563,-0.00205966,0.4780085,1.734641
0,18.679154,15.438,15.09984,33.44330,-0.1389111,0.3381612373,-2.43437194,0.4780085,1.734641
0,21.361138,15.342,14.77717,33.44542,-0.1404613,0.5648314560,-4.02126157,0.4780085,1.734641


In [4]:
pseudo_invert<-function(ds,ps,strat,strat_depth){
   H <- make_inv_matrix(ds,ps,strat,strat_depth)
   system.time(approx <- tappered_least_square(H,ds$dz))
   xl <- length(approx$x_hat)
   ps$a  <- approx$x_hat[1:(xl/2),]
   ps$b  <- approx$x_hat[(xl/2 + 1) :xl,]
   return(ps)
}

#ps_approx <- pseudo_invert(ds,ps,strat,depth)

In [5]:
plot_kspace_gg <- function(ps){
    p <- ggplot(ps,aes(x=kx*1000,y=ky*1000,size=mag(a,b),color=mag(a,b))) + 
                geom_point() + 
                labs(x='East-West (cpk)', y='North-South (cpk)') +
                scale_color_gradient(low = "blue", high = "red")  +
                theme(panel.background = element_rect(fill = 'blue', colour = 'white'))
     return (p)
}

plot_kspace_gg(ps_approx)
#ps_approx$mag <- sqrt(ps_approx$a**2 + ps_approx$b**2)
#plot(ps_approx$kx,ps_approx$mag)

ERROR: Error in ggplot(ps, aes(x = kx * 1000, y = ky * 1000, size = mag(a, b), : object 'ps_approx' not found


In [ ]:
source('../src/inversion.r')
sample_times <- seq(1*24*3600,9*24*3600,1800)
ds_fut <- ds_sbc[ds_sbc$t %in% sample_times,]
ds_fut <- make_forecast(ds_fut,ps_approx,strat,depth)
rms_error <- forecast_error(ds_fut)

In [ ]:
convert_to_date <- function(t){
     ret<-as.POSIXct(t + 1473369000 ,origin="1970-01-01")
     return(ret)
}

samp <- ds_fut[sample(nrow(ds_fut),2), ]
samp <- samp[1,]
out = subset(ds_fut, x%in%samp$x & y%in%samp$y & z%in%samp$z)
#time <- as.POSIXct(out$t + 1473369000 ,origin="1970-01-01")
time <- convert_to_date(out$t)
plot(time,out$dz,type='l',lty=4)
lines(time,out$d_est)
title(str(samp$z)  )

In [ ]:
plot(convert_to_date(unique(ds_fut$t)),rms_error,type='l')

# Modal Analysis

In [ ]:
library('feather')
source('../src/inversion.r')
source('../src/modes.r')

#Ocean profile
stratdf <- read_feather('sbc_strat.fthr')
strat   <- stratdf$strat
depth   <- stratdf$depth

#Subsamples Stratification
ss <- seq(1,length(strat),5)
strat <- strat[ ss ]
depth <- depth[ ss ]

#Data subsample
m1 <- subset(ds_sbc,x==unique(ds_sbc$x)[1] & y==unique(ds_sbc$y)[1] & dz < 50)
m2 <- subset(ds_sbc,x==unique(ds_sbc$x)[2] & y==unique(ds_sbc$y)[2])
sample_times = seq(0,9*24*3600,1*3600)
m1 <- m1[m1$t %in% sample_times,]
m2 <- m2[m2$t %in% sample_times,]


#Estimate Parameters
estimate_params <- function(ds,ps,strat,depth){
    ps_approx <- pseudo_invert(ds,ps,strat,depth)
    ds_fut <- make_forecast(ds,ps_approx,strat,depth)
    return (list(ds=ds_fut,ps=ps_approx))
    
}

M2 <- 0.0805/3600
headings  <- c(0)

#Case 1
modes <- c(1)
omega <- c(M2)
k  <- generate_wavenumbers(depth,strat,omega,modes)
ps <- generate_parameter_space(omega,modes,k,headings) 

est1_case1 <- estimate_params(m1,ps,strat,depth)
est2_case1 <- estimate_params(m2,ps,strat,depth)

#Case 2
modes <- seq(1,50)
omega <- c(M2) 
k  <- generate_wavenumbers(depth,strat,omega,modes)
ps <- generate_parameter_space(omega,modes,k,headings) 

est1_case2 <- estimate_params(m1,ps,strat,depth)
est2_case2 <- estimate_params(m2,ps,strat,depth)

#Case 3
modes <- seq(1,100)
omega <- coh_freqs/3600
k  <- generate_wavenumbers(depth,strat,omega,modes)
ps <- generate_parameter_space(omega,modes,k,headings) 

est1_case3 <- estimate_params(m1,ps,strat,depth)
est2_case3 <- estimate_params(m2,ps,strat,depth)

#head(ds_fut)
#head(ps_approx)



In [ ]:
#write_feather(est1[['ds']],'50-mode-est1.fthr')
#write_feather(est2[['ds']],'50-mode-est2.fthr')

In [ ]:
plot(est1[['ps']]$a)
points(est2[['ps']]$a,col='red')

In [ ]:
source('../src//inversion.r')
convert_to_date <- function(t){
     ret<-as.POSIXct(t + 1473369000 ,origin="1970-01-01")
     return(ret)
}


plot_fit_rs <- function(ests,samples){
    colors <- c('red','green','blue')
    rs = sample(samples$t,1)
    
    plot(samples[samples$t == rs,]$dz, samples[samples$t == rs,]$z, ylim = rev(range(samples[samples$t == rs,]$z)),
         type='l',lty=2,
         xlab='Displacement(m)',
         ylab='Depth (m)') 
    
    i<-1
    for ( est in ests ){
        ds_fut <- est[['ds']]
        lines(ds_fut[ds_fut$t == rs,]$d_est, ds_fut[ds_fut$t == rs,]$z,
          ylim = rev(range(samples[samples$t == rs,]$z)),lty=1,col=colors[i] )
        i<- i + 1
    } 
    title('Random Sample')
    title(sub=convert_to_date(rs))

}

plot_rms_error <- function(ests,samples){
    colors <- c('red','green','blue')
   
    i <- 1
    for ( est in ests ){
        ds_fut <- est[['ds']]
        rms_error <- forecast_error(ds_fut,'z')
        zz <- unique(ds_fut$z)
       
        if (i == 1) {
         plot(rms_error,zz,ylim = rev(range(zz) ), 
             xlab='RMS Error',
             ylab='Depth (m)',
             type='l',
             col=colors[i])
            
        }else{
        lines(rms_error,zz,ylim = rev(range(zz) ),
         xlab='RMS Error (m)',
         ylab='Depth (m)',
         col=colors[i])
        }
     i<- i + 1
    }
   title('Mooring RMS Error')
}    

par(mfrow=c(1,2))
plot_fit_rs( list(est1_case1,est1_case2,est1_case3), m1)
plot_rms_error( list(est1_case1,est1_case2,est1_case3), m1)
legend("topright", legend=c("Obs.", "M2M01","M2M50","CFM50"),
       col=c("black","red","green", "blue"), lty=c(2,1,1,1), cex=0.8)


In [ ]:
library('reshape2')
erm <- est1_case2[['ds']]
erm$error <- erm$dz
#erm$error <- sqrt((erm$dz-erm$d_est)**2)/abs(erm$dz)
#erm$error[abs(erm$error)>50,] <- 0
erm <- acast(erm,t~z,value.var='error')
filled.contour(x = convert_to_date(as.numeric(rownames(erm))),
               y = as.numeric(colnames(erm)),
               z = erm,
               zlim = c(-max(abs(erm)),max(abs(erm))),
               ylim=rev(range( as.numeric(colnames(erm)))))
zlimit <- max(abs(erm))
title('Observation')

In [ ]:

erm <- est1_case2[['ds']]
erm$error <- erm$d_est
erm <- acast(erm,t~z,value.var='error')
filled.contour(x = convert_to_date(as.numeric(rownames(erm))),
               y = as.numeric(colnames(erm)),
               z = erm,
               zlim = c(-zlimit,zlimit),
               ylim=rev(range( as.numeric(colnames(erm)))))
title('Estimate')

In [ ]:

erm <- est1_case2[['ds']]
erm$error <- erm$dz- erm$d_est 
erm <- acast(erm,t~z,value.var='error')
filled.contour(x = convert_to_date(as.numeric(rownames(erm))),
               y = as.numeric(colnames(erm)),
               z = erm,
               zlim = c(-zlimit,zlimit),
               ylim=rev(range( as.numeric(colnames(erm)))))
title('Anomaly from Model (meters)')
#dev.off()

In [ ]:
#library('akima')
#library(plyr)
#dsr<- subset(est1_case3[['ds']],select=c('t','z','d_est'))
#dsr<-rename(dsr, c("t"="x","z"="y","d_est"="z"))
#
#dso<- subset(est1_case2[['ds']],select=c('t','z','d_est'))
#dso<- rename(dso, c("t"="x","z"="y","d_est"="z"))
#
#par(mfrow=c(1,2))
#levelplot(z~x*y,data=dsr, panel=panel.levelplot.raster,interpolate=T ,ylim=rev(range(ds$y)))
#levelplot(z~x*y,data=dso, panel=panel.levelplot.raster,interpolate=T ,ylim=rev(range(ds$y)))

In [ ]:

p1=atan2( est1[['ps']]$b,est1[['ps']]$a)
p2=atan2( est2[['ps']]$b,est2[['ps']]$a)
print( 180*(p1-p2)/pi )

In [ ]:
plot(est1_case3[['ps']]$k,mag(est1_case3[['ps']]$a,est1_case3[['ps']]$b))

In [ ]:
params <- est1_case3[['ps']]
params$mag <- mag(est1_case3[['ps']]$a,est1_case3[['ps']]$b)
qplot(omega, modes, colour = mag/max(mag), data = params) + 
     scale_colour_gradient(name = "category", low = "white", high = "black")+
     theme_bw()